In [41]:
import numpy as np
import mlflow
import mlflow.keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import pickle
import os
import pandas as pd

In [3]:
df = pd.read_csv("C:/Users/sofia/Downloads/Repositorios analítica/analítica/proyecto2/Datos/Datos2011Limipios.csv")
feature_columns = ['cole_depto_ubicacion', 'cole_jornada', 
                      'cole_naturaleza', 'fami_educacionmadre', 'fami_educacionpadre', 
                      'fami_estratovivienda']

Transformación de los datos de entrada para el modelo

In [45]:
def transform_new_data(new_data, encoders, reference_columns):
    import pandas as pd
    from sklearn.preprocessing import LabelEncoder

    new_data_encoded = new_data.copy()

    # Aplicar LabelEncoder
    for col, le in encoders.items():
        if col in new_data_encoded.columns:
            new_data_encoded[col] = new_data_encoded[col].astype(str)
            new_data_encoded[col] = new_data_encoded[col].apply(
                lambda x: x if x in le.classes_ else le.classes_[0]
            )
            new_data_encoded[col] = le.transform(new_data_encoded[col])

    # Crear dummies
    multi_columns = [col for col in new_data.columns if col not in encoders.keys()]
    new_data_encoded = pd.get_dummies(new_data_encoded, columns=multi_columns, drop_first=False, dtype=int)

    # Verificar duplicados
    print("Columnas duplicadas en feature_columns:", pd.Series(reference_columns).duplicated().any())
    print("Columnas duplicadas en new_data_encoded:", new_data_encoded.columns.duplicated().any())

    # Eliminar duplicados si es necesario
    if new_data_encoded.columns.duplicated().any():
        print("Columnas duplicadas encontradas y eliminadas:", new_data_encoded.columns[new_data_encoded.columns.duplicated()])
        new_data_encoded = new_data_encoded.loc[:, ~new_data_encoded.columns.duplicated()]

    # Asegurar columnas
    for col in reference_columns:
        if col not in new_data_encoded.columns:
            new_data_encoded[col] = 0

    new_data_encoded = new_data_encoded.reindex(columns=reference_columns, fill_value=0)
    new_data_encoded = new_data_encoded.astype('float32')

    return new_data_encoded

In [ ]:
# Ejemplo
new_data = df[feature_columns].iloc[15:16]

columns_info_path = "C:/Users/sofia/Downloads/Repositorios analítica/analítica/proyecto2/Modelamiento/modelos_guardados/columns_info.pkl"

with open(columns_info_path, "rb") as f:
        columns_info = pickle.load(f)

new_data_processed = transform_new_data(
        new_data[columns_info['categorical_columns'] + ['fami_estratovivienda']], 
        columns_info['encoders'], 
        columns_info['feature_columns']
    )

Columnas duplicadas en feature_columns: False
Columnas duplicadas en new_data_encoded: True
Columnas duplicadas encontradas y eliminadas: Index(['fami_estratovivienda_Estrato 3', 'fami_estratovivienda_Estrato 3',
       'fami_estratovivienda_Estrato 3'],
      dtype='object')


In [47]:
columns_info

{'feature_columns': ['periodo',
  'estu_tipodocumento',
  'cole_area_ubicacion',
  'cole_bilingue',
  'cole_calendario',
  'cole_caracter',
  'cole_genero',
  'cole_mcpio_ubicacion',
  'cole_naturaleza',
  'cole_sede_principal',
  'estu_depto_presentacion',
  'estu_depto_reside',
  'estu_estudiante',
  'estu_genero',
  'estu_mcpio_presentacion',
  'estu_mcpio_reside',
  'estu_nacionalidad',
  'estu_pais_reside',
  'estu_privado_libertad',
  'fami_tienecomputador',
  'fami_tieneinternet',
  'desemp_ingles',
  'punt_ingles',
  'cole_depto_ubicacion_ANTIOQUIA',
  'cole_depto_ubicacion_ARAUCA',
  'cole_depto_ubicacion_ATLANTICO',
  'cole_depto_ubicacion_BOGOTA',
  'cole_depto_ubicacion_BOGOTÁ',
  'cole_depto_ubicacion_BOLIVAR',
  'cole_depto_ubicacion_BOYACA',
  'cole_depto_ubicacion_CALDAS',
  'cole_depto_ubicacion_CAQUETA',
  'cole_depto_ubicacion_CASANARE',
  'cole_depto_ubicacion_CAUCA',
  'cole_depto_ubicacion_CESAR',
  'cole_depto_ubicacion_CHOCO',
  'cole_depto_ubicacion_CORDOBA',
 

In [48]:
new_data

,cole_depto_ubicacion,cole_jornada,cole_naturaleza,fami_educacionmadre,fami_educacionpadre,fami_estratovivienda
15,BOGOTA,COMPLETA,NO OFICIAL,Educación profesional completa,Educación profesional completa,Estrato 3


In [49]:
for j in new_data_processed.columns:
    if new_data_processed[j].iloc[0] > 0:
        print(j, new_data_processed[j].iloc[0])

cole_depto_ubicacion_BOGOTA 1.0
fami_estratovivienda_Estrato 3 1.0


Predicción según el modelo para el perfil

In [32]:
def load_best_model_and_predict( modelo, columns_info_path, new_data_path=None, new_data_df=None):
    """
    Carga el mejor modelo y hace predicciones en nuevos datos
    """
    # Cargar modelo y información de columnas
    model = tf.keras.models.load_model(modelo)
    
    with open(columns_info_path, "rb") as f:
        columns_info = pickle.load(f)
    
    if new_data_path:
        new_data = pd.read_csv(new_data_path)
    elif new_data_df is not None:
        new_data = new_data_df.copy()
    else:
        # Usar datos de test como ejemplo
        new_data = df[feature_columns].iloc[15:16]  # Primera fila como ejemplo
    
    # Procesar nuevos datos
    new_data_processed = transform_new_data(
        new_data[columns_info['categorical_columns'] + ['fami_estratovivienda']], 
        columns_info['encoders'], 
        columns_info['feature_columns']
    )
    
    # Hacer predicciones
    predictions_prob = model.predict(new_data_processed)
    predictions_binary = (predictions_prob > 0.5).astype(int)
    
    # Convertir a etiquetas legibles
    predictions_labels = ['exitoso' if pred == 1 else 'no_exitoso' for pred in predictions_binary.flatten()]
    
    return {
        'probabilidades': predictions_prob.flatten(),
        'predicciones_binarias': predictions_binary.flatten(),
        'predicciones_etiquetas': predictions_labels,
        'datos_procesados': new_data_processed
    }

In [50]:
# Ejemplo

resultados = load_best_model_and_predict(modelo="C:/Users/sofia/Downloads/Repositorios analítica/analítica/proyecto2/Modelamiento/ejemplo.keras",
                                              columns_info_path="C:/Users/sofia/Downloads/Repositorios analítica/analítica/proyecto2/Modelamiento/columns_info_ej.pkl")
print("Predicciones en datos de ejemplo:")
for i, (prob, etiqueta) in enumerate(zip(resultados['probabilidades'], resultados['predicciones_etiquetas'])):
        print(f"  Ejemplo {i+1}: {etiqueta} (probabilidad: {prob:.4f})")

Columnas duplicadas en feature_columns: False
Columnas duplicadas en new_data_encoded: True
Columnas duplicadas encontradas y eliminadas: Index(['fami_estratovivienda_Estrato 3', 'fami_estratovivienda_Estrato 3',
       'fami_estratovivienda_Estrato 3'],
      dtype='object')
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step
Predicciones en datos de ejemplo:
  Ejemplo 1: no_exitoso (probabilidad: 0.0000)


In [38]:
df[feature_columns].iloc[15:16]

,cole_depto_ubicacion,cole_jornada,cole_naturaleza,fami_educacionmadre,fami_educacionpadre,fami_estratovivienda
15,BOGOTA,COMPLETA,NO OFICIAL,Educación profesional completa,Educación profesional completa,Estrato 3


In [39]:
for j in resultados['datos_procesados'].columns:
    if new_data_processed[j].iloc[0] > 0:
        print(j, new_data_processed[j].iloc[0])

cole_depto_ubicacion_BOGOTA 1.0
fami_estratovivienda_Estrato 3 1.0


In [40]:
resultados['probabilidades']

array([4.412931e-15], dtype=float32)